In [1]:
from sqlalchemy import create_engine
from sqlalchemy.sql.ddl import CreateSchema
from tablas_mfm import *
from tablas_origen import *
import numpy as np
import pandas as pd
import generador_filas as gf
from tablas_generacion import *
import datetime
import funciones_ruido as fr

In [2]:
#Devuelve una tabla con nombre "nombre"
def string_to_table(nombre):
    for tb in metadata.sorted_tables:
        if tb.name == nombre:
            return tb
    raise Exception("la tabla "+nombre+" no existe")


In [3]:
#Copia la tabla "nombreTabla" de la BBDD "dbO" a la BBDD "dbD"
def copiarTabla(dbO,dbD, nombreTabla):
    if not isinstance(nombreTabla,Table) :
        raise Exception("la tabla "+nombreTabla+" no existe")
    else :
        result = dbO.execute(nombreTabla.select())
        column=nombreTabla.columns.keys()
        filas=[]
        filasmax=10000
        num_filas=0
        for r in result:
            fila = {}
            for i in range(len(column)):
                fila.update({column[i] : r[i]})
            filas.append(fila)
            num_filas+=1
            if num_filas > filasmax:
                insercion = nombreTabla.insert().values(filas)
                dbD.execute(insercion)
                filas = []
                num_filas=0
        result.close()
        insercion = nombreTabla.insert().values(filas)
        dbD.execute(insercion)


In [4]:
#Crea las conexiones con las BBDD origen y destino
db_destino_string = ""
db_origen_string = ""

dbDestino = create_engine(db_destino_string)
dbOrigen = create_engine(db_origen_string)

dbDes=dbDestino.connect()
dbOr=dbOrigen.connect()

In [5]:
#Crea el esquema para las tablas en la BBDD destino

if not dbDes.dialect.has_schema(dbDes, 'mfm'):
    dbDes.execute(CreateSchema('mfm'))

In [6]:
#Crea las tablas en la BBDD destino
metadata.create_all(dbDes)


In [7]:
#Copia las tablas que sean diccionarios a la base de datos destino
dfs = pd.read_excel('Tablas.xlsx',engine='openpyxl',header=None)
tablas_mfm=dfs.values
for tabla in tablas_mfm:
    if tabla[1] == 'D' :
        copiarTabla(dbOr,dbDes,string_to_table(tabla[0]))



In [8]:
#Cuenta el numero de sectores
tipo_zbs=["RU","UR","CA"]

In [9]:
res = dbOr.execute(text("select count(*) from mfm.mf_sector"))
nSectores = res.first()[0]
res.close()

sectores = np.zeros([len(tipo_zbs),nSectores])

In [10]:
#Cuenta el numero de zbs segun su tipo que tiene cada sector
q1="select * from (select sector_id , count(*) from mfm.mf_zbs where tipo = '"
q2="' group by sector_id) as zonas order by sector_id;"

for i in range(0,len(tipo_zbs)):
    res= dbOr.execute(text(q1+tipo_zbs[i]+q2))
    for r in res:
        sectores[i,r[0]-1]=r[1]
    res.close()

print(sectores)

[[10. 12.  3. 14.  8. 14. 11.  5.]
 [ 1.  3.  0.  2.  2.  0.  1.  0.]
 [ 3.  0. 18.  6.  0.  2.  0.  8.]]


In [11]:
#Genera los sectores con sus zbs
NUMERO_DE_SECTORES_A_GENERAR=5

In [12]:
sectores_generados = np.random.multivariate_normal(np.mean(sectores,axis=1), np.cov(sectores), NUMERO_DE_SECTORES_A_GENERAR)
sectores_generados[sectores_generados<0]=0
sectores_generados=np.round(sectores_generados)
print(sectores_generados)

[[12.  1.  2.]
 [ 7.  1.  0.]
 [ 8.  2.  8.]
 [ 8.  0.  5.]
 [14.  3.  0.]]


In [13]:
#Cuenta el número zbs totales segun su tipo
q1="select count(*) from mfm.mf_zbs where tipo ='"
q2="';"
nTipo_zbs=[]
for i in range(0,len(tipo_zbs)):
    res= dbOr.execute(text(q1+tipo_zbs[i]+q2))
    nTipo_zbs.append(res.first()[0])
    res.close()
print(nTipo_zbs)

[77, 9, 37]


In [14]:
tipo_AP = ["MEDICINA FAMILIA", "PEDIATRIA AP"]

In [15]:
#Cuenta el numero de cias de AP segun su tipo en cada tipo de zbs
q1="select mz.code, coalesce(_count,0)  from (select zbs_cd as z , count(*) as _count from mfm.mf_cias_v2 where speciality_st = '"
q2="' group by zbs_cd) as s right join mfm.mf_zbs as mz on s.z=mz.code where mz.tipo='"
q3="' order by mz.code;"
cias=[]
for i in range(0,len(tipo_zbs)):
    cias_por_tipo = np.zeros([len(tipo_AP),nTipo_zbs[i]])
    for j in range(0,len(tipo_AP)):
        res= dbOr.execute(text(q1+tipo_AP[j]+q2+tipo_zbs[i]+q3))
        k=0
        for r in res:
            cias_por_tipo[j,k]=r[1]
            k+=1
        res.close()
    cias.append(cias_por_tipo)

In [16]:
#Genero los cias de AP en las zbs creadas
cias_generados=[]
for i in range(0,len(tipo_zbs)):
    cias_generado=np.random.multivariate_normal(np.mean(cias[i],axis=1), np.cov(cias[i]),np.sum(sectores_generados,axis=0).astype(int)[i] )
    cias_generado[cias_generado<0]=0
    cias_generado=np.round(cias_generado)
    cias_generados.append(cias_generado)
#print(cias_generados)


In [17]:
#Inserto en la BBDD destino los sectores,zbs y cias generados
i=0
idsZ=0
idsC=0
indice_tipos_cias=np.zeros([len(tipo_zbs)]).astype(int)
for sector in sectores_generados:
    s = gf.generar_sector(i)
    ins_sec = t_mf_sector.insert().values(s)
    dbDes.execute(ins_sec)
    for k in range(0,len(tipo_zbs)):
        j=0
        while j < sector[k]:
            zbs = gf.generar_zbs(idsZ,tipo_zbs[k],s)
            dbDes.execute(t_mf_zbs.insert().values(zbs))
            cias_g=cias_generados[k][indice_tipos_cias[k]]
            indice_tipos_cias[k]+=1
            l=0
            while l < len(tipo_AP):
                sp={"code": l, "st": tipo_AP[l]}
                m=0
                while m < cias_g[l]:
                    ci = gf.generar_cias(idsC,sp,s,zbs)
                    dbDes.execute(t_mf_cias_v2.insert().values(ci))
                    idsC+=1
                    m+=1
                l+=1
            idsZ+=1
            j+=1
    i+=1

In [18]:
#Crea la tabla con la equivalencia entre los antiguos y nuevos usuarios
if not dbDes.dialect.has_schema(dbDes, 'gn'):
    dbDes.execute(CreateSchema('gn'))
metadata_g.create_all(dbDes)

In [19]:
query="select * from (select id, sexo, nacimiento_dt, zbs_cd, sector_cd from mfm.mf_usuarios where active=true) as us join"\
      "(select id, tipo from mfm.mf_zbs) as zb on zb.id=us.zbs_cd;"
res = dbOr.execute(text(query))
i=0
for r in res:

    #dias desplazados
    dias=np.random.randint(0,365)
    nacimiento=r[2]-datetime.timedelta(days=dias)

    #calcula edad
    edad=datetime.date.today().year-nacimiento.year
    if nacimiento.month>datetime.date.today().month:
        edad=edad-1
    elif nacimiento.month==datetime.date.today().month \
        and nacimiento.day>=datetime.date.today().day:
        edad=edad-1
    #decide entre medico de familia y pediatra segun edad (mayor o menor de 14)
    if edad>=14:
        u_AP=0
    else:
        u_AP=1


    #selecciona el tipo de zbs
    tipo=tipo_zbs.index(r[6])
    u = np.random.randint(1,10)
    aux=0
    if u<8:
        aux=0
    elif u<9:
        aux=1
    else:
        aux=2

    tipo_n=(tipo+aux) % 3

    repito=True
    while repito:
    #selecciona nuevo sector
        noseleccionado=True
        sector=np.random.randint(0,NUMERO_DE_SECTORES_A_GENERAR-1)
        while noseleccionado:
            query="select count(*) from mfm.mf_zbs where sector_id="+str(sector)+" and tipo='"+tipo_zbs[tipo_n]+"';"
            if dbDes.execute(text(query)).first()[0]==0:
                sector=(sector+1) % NUMERO_DE_SECTORES_A_GENERAR
            else:
                noseleccionado=False

    #selecciona zbs
        query="select min(cast(id as int)),max(cast(id as int)) from mfm.mf_zbs where sector_id="+str(sector)+" and tipo='"+tipo_zbs[tipo_n]+"';"
        res_zbs=dbDes.execute(text(query))
        res_zbs_id=res_zbs.first()
        minimo=int(res_zbs_id[0])
        maximo=int(res_zbs_id[1])
        if minimo==maximo:
            zbs=res_zbs_id[0]
            query="select count(*) from mfm.mf_cias_v2 where zbs_cd='"+str(zbs)+"' and speciality_st='"+tipo_AP[u_AP]+"';"
            res_hay_cias=dbDes.execute(text(query))
            res_hay=res_hay_cias.first()
            if res_hay[0] > 0:
                repito = False
        else:
            noListo=True
            while noListo:
                zbs=np.random.randint(res_zbs_id[0],res_zbs_id[1])
                query="select count(*) from mfm.mf_cias_v2 where zbs_cd='"+str(zbs)+"' and speciality_st='"+tipo_AP[u_AP]+"';"
                res_hay_cias=dbDes.execute(text(query))
                res_hay=res_hay_cias.first()
                if res_hay[0] > 0:
                    noListo = False
                    repito = False


    #selecciona cias
    query="select min(cast(code as int)), max(cast(code as int)) from mfm.mf_cias_v2 where zbs_cd='"+str(zbs)+"' and speciality_st='"+tipo_AP[u_AP]+"';"
    res_cias=dbDes.execute(text(query))
    res_cias_id=res_cias.first()
    minimo=int(res_cias_id[0])
    maximo=int(res_cias_id[1])
    if minimo==maximo:
        cias=res_cias_id[0]
    else:
        cias=np.random.randint(res_cias_id[0],res_cias_id[1])


    #inserta el nuevo usuario en la BD destino e inserta la equivalencia con el antiguo usuario
    us = gf.generar_usuario(i,r[1],nacimiento,zbs,sector,cias)
    ins_us = t_mf_usuarios.insert().values(us)
    eq = gf.generar_eq(r[0],i,dias)
    ins_eq = t_gn_equivalente.insert().values(eq)
    dbDes.execute(ins_us)
    dbDes.execute(ins_eq)
    i+=1
res.close()

In [20]:
#Crea las tablas del esquema origen
if not dbDes.dialect.has_schema(dbDes, 'origen'):
    dbDes.execute(CreateSchema('origen'))
metadata_or.create_all(dbDes)

In [21]:
query="select * from gn.gn_equivalente;"
usuario=dbDes.execute(text(query))
u_eq=np.array(usuario.fetchall())

In [22]:
#Inserto datos en la tabla diag_ap
query="select * from origen.diag_ap;"
res = dbOr.execute(text(query))
for r in res:
    #print(r)
    u=u_eq[np.where(u_eq[:,0] == r[0])[0]]
    if u.size==0:
        #print("usuario no existe")
        pass
    elif np.random.randint(0, 100)<=98:
        id=u[0][1]
        ruido=fr.ruido()
        fecha_i=fr.correrFecha(r[4],(u[0][2]+ruido).item())
        existe_fecha_f=False
        if r[5] is not None:
            fecha_f=fr.correrFecha(r[5],(u[0][2]+ruido).item())
            existe_fecha_f=True
        else:
            fecha_f=0
        episodio=fr.aplicarHash(r[1])
        fila_diag_ap=gf.generar_diag_ap(id.item(),episodio,r[2],r[3],fecha_i,fecha_f,existe_fecha_f)
        ins_diag_ap=t_diag_ap.insert().values(fila_diag_ap)
        dbDes.execute(ins_diag_ap)

In [23]:
#Inserto datos en la tabla visits_pc
query="select * from origen.visits_pc;"
res = dbOr.execute(text(query))
for r in res:
    #print(r)
    u=u_eq[np.where(u_eq[:,0] == r[0])[0]]
    if u.size==0:
        #print("usuario no existe")
        pass
    elif np.random.randint(0, 100)<=98:
        id=u[0][1]
        ruido=fr.ruido()
        fecha=fr.correrFecha(r[4],(u[0][2]+ruido).item())
        fila_visits_pc=gf.generar_visits_pc(id.item(),r[1],r[2],fecha,r[5],r[6],r[7])
        #print(fila_visits_pc)
        ins_visits_pc=t_visits_pc.insert().values(fila_visits_pc)
        dbDes.execute(ins_visits_pc)

In [24]:
#Inserto datos en la tabla visits_sc
query="select * from origen.visits_sc;"
res = dbOr.execute(text(query))
for r in res:
    #print(r)
    u=u_eq[np.where(u_eq[:,0] == r[0])[0]]
    if u.size==0:
        #print("usuario no existe")
        pass
    elif np.random.randint(1, 100)<=98:
        id=u[0][1]
        ruido=fr.ruido()
        fecha=fr.correrFecha(r[5],(u[0][2]+ruido).item())
        if np.random.randint(1, 10)<=9:
            query="select sector_cd from mfm.mf_usuarios where id='"+str(id.item())+"' ;"
            res_hospital = dbDes.execute(text(query))
            res_h_a=np.array(res_hospital.fetchall())
            id_hospital=(res_h_a[0][0]).item()
            hospital="HOSPITAL "+str(id_hospital)
        else:
            id_hospital=np.random.randint(0,NUMERO_DE_SECTORES_A_GENERAR)
            hospital="HOSPITAL "+str(id_hospital)

        fila_visits_sc=gf.generar_visits_sc(id.item(),id_hospital,hospital,fecha,r[6],
                                            r[7],r[8],r[9],r[10],r[11],r[12],r[13],r[14],
                                            r[15])
        ins_visits_sc=t_visits_sc.insert().values(fila_visits_sc)
        dbDes.execute(ins_visits_sc)

In [25]:
#Inserto datos en la tabla visits_er
query="select * from origen.visits_er;"
res = dbOr.execute(text(query))

for r in res:
    #print(r)
    u=u_eq[np.where(u_eq[:,0] == r[0])[0]]
    if u.size==0:
        #print("usuario no existe")
        pass
    elif np.random.randint(1, 100)<=98:
        datos=[]
        id=u[0][1]
        datos.append(id.item())
        datos.extend(r[1:4])
        ruido=fr.ruido()
        ruidoHora=fr.ruidoHora()
        fecha=fr.correrFecha(r[4],(u[0][2]+ruido).item())-ruidoHora
        datos.append(fecha)
        if np.random.randint(1, 10)<=9:
            query="select sector_cd from mfm.mf_usuarios where id='"+str(id.item())+"' ;"
            res_hospital = dbDes.execute(text(query))
            res_h_a=np.array(res_hospital.fetchall())
            id_hospital=(res_h_a[0][0]).item()
            hospital="HOSPITAL "+str(id_hospital)
        else:
            id_hospital=np.random.randint(0,NUMERO_DE_SECTORES_A_GENERAR)
            hospital="HOSPITAL "+str(id_hospital)
        datos.append(id_hospital)
        datos.append(hospital)
        datos.extend(r[7:22])
        if r[22]:
            datos.append(fr.correrFecha(r[22],(u[0][2]+ruido).item())-ruidoHora)
        else:
            datos.append(None)
        datos.extend(r[23:25])
        datos.append(fr.correrFecha(r[25],(u[0][2]+ruido).item())-ruidoHora)
        datos.append(fr.correrFecha(r[26],(u[0][2]+ruido).item())-ruidoHora)
        datos.append(fr.correrFecha(r[27],(u[0][2]+ruido).item())-ruidoHora)
        datos.append(r[28])

        fila_visits_er=gf.generar_fila(t_visits_er.columns.keys(),datos)
        ins_visits_er=t_visits_er.insert().values(fila_visits_er)
        dbDes.execute(ins_visits_er)

In [26]:
#Inserto datos en la tabla cmbd
query="select * from origen.cmbd;"
res = dbOr.execute(text(query))

for r in res:
    #print(r)
    u=u_eq[np.where(u_eq[:,0] == r[0])[0]]
    if u.size==0:
        #print("usuario no existe")
        pass
    elif np.random.randint(1, 100)<=98:
        datos=[]
        id=u[0][1]
        datos.append(id.item())
        if np.random.randint(1, 10)<=9:
            query="select sector_cd from mfm.mf_usuarios where id='"+str(id.item())+"' ;"
            res_hospital = dbDes.execute(text(query))
            res_h_a=np.array(res_hospital.fetchall())
            id_hospital=(res_h_a[0][0]).item()
            hospital="HOSPITAL "+str(id_hospital)
        else:
            id_hospital=np.random.randint(0,NUMERO_DE_SECTORES_A_GENERAR)
            hospital="HOSPITAL "+str(id_hospital)
        datos.append(id_hospital)
        datos.append(hospital)
        datos.append(fr.correrFecha(r[3],(u[0][2]+ruido).item()))
        datos.append(r[4])
        datos.append(fr.correrFecha(r[5],(u[0][2]+ruido).item()))
        datos.extend(r[6:55])
        if r[55]:
            datos.append(fr.correrFecha(r[55],(u[0][2]+ruido).item()))
        else:
            datos.append(None)
        datos.extend(r[56:69])
        fila_cmbd=gf.generar_fila(t_cmbd.columns.keys(),datos)
        ins_cmbd=t_cmbd.insert().values(fila_cmbd)
        dbDes.execute(ins_cmbd)

In [27]:
#Inserto datos en la tabla phar_presc
query="select * from origen.phar_presc;"
res = dbOr.execute(text(query))

for r in res:
    #print(r)
    u=u_eq[np.where(u_eq[:,0] == r[0])[0]]
    if u.size==0:
        #print("usuario no existe")
        pass
    elif np.random.randint(1, 100)<=98:
        datos=[]
        id=u[0][1]
        datos.append(id.item())
        ruido=fr.ruido()
        fecha=fr.correrFecha(r[3],(u[0][2]+ruido).item())
        datos.append(fecha.year)
        datos.append(fecha.year*100+fecha.month)
        datos.append(fecha)
        datos.append(fr.aplicarHash(r[4]))
        datos.append(None)
        datos.extend(r[6:8])
        datos.append(fr.correrFecha(r[8],(u[0][2]+ruido).item()))
        datos.append(fr.correrFecha(r[9],(u[0][2]+ruido).item()))
        datos.extend(r[10:26])
        if r[26]:
            datos.append(fr.aplicarHash(r[26]))
        else:
            datos.append(None)
        datos.append(r[27])

        query_cias="select cias_cd from mfm.mf_usuarios where id='"+str(id.item())+"';"
        res_cias = dbDes.execute(text(query_cias))
        datos.append(res_cias.first()[0])

        datos.extend(r[29:31])
        datos.append(fr.aplicarHash(r[31]))
        datos.append(fr.aplicarHash(r[32]))

        fila_phar=gf.generar_fila(t_phar_presc.columns.keys(),datos)
        ins_phar=t_phar_presc.insert().values(fila_phar)
        dbDes.execute(ins_phar)

In [28]:
dbDes.close()
dbDestino.dispose()
dbOr.close()
dbOrigen.dispose()